In [1]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("StructuredKafkaWordCount") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.4') \
    .config("spark.sql.shuffle.partitions", 4) \
    .master("local[*]") \
    .getOrCreate()

spark

:: loading settings :: url = jar:file:/usr/local/spark-3.2.4-bin-hadoop2.7/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8f5bd0b5-07dd-424a-91fa-48ed9657d95d;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.4 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central

In [2]:
bootstrapServers = "kafka1:19091,kafka2:19092,kafka3:19093"
subscribeType = "subscribe"
topics = "kafka-word-count-example"

# Create DataSet representing the stream of input lines from kafka
lines = spark\
    .readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers", bootstrapServers)\
    .option(subscribeType, topics)\
    .load()\
    .selectExpr("CAST(value AS STRING)")

In [ ]:
checkpointDir = '/home/jovyan/work/utils/checkpoint'

# Split the lines into words
words = lines.select(
    # explode turns each item in an array into a separate row
    explode(
        split(lines.value, ' ')
    ).alias('word')
)

# Generate running word count
wordCounts = words.groupBy('word').count()

# Start running the query that prints the running counts to the console
query = (wordCounts
    .selectExpr(
        "CAST(word AS string) AS key",
        "CAST(count AS string) AS value")
    .writeStream
    .trigger(processingTime='3 seconds')
    .format("kafka")
    .option("kafka.bootstrap.servers", bootstrapServers)
    .option("topic", "wordCounts")
    .option("checkpointLocation", checkpointDir)
    .outputMode('update')
    .start())

query.awaitTermination()

24/02/01 11:01:09 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/02/01 11:01:25 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 15392 milliseconds
24/02/01 11:01:41 WARN NetworkClient: [Producer clientId=producer-1] Error while fetching metadata with correlation id 1 : {wordCounts=LEADER_NOT_AVAILABLE}
24/02/01 11:01:41 WARN NetworkClient: [Producer clientId=producer-1] Error while fetching metadata with correlation id 3 : {wordCounts=LEADER_NOT_AVAILABLE}
24/02/01 11:01:41 WARN NetworkClient: [Producer clientId=producer-1] Error while fetching metadata with correlation id 4 : {wordCounts=LEADER_NOT_AVAILABLE}
24/02/01 11:01:41 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 5832 milliseconds
24/02/01 11:02:51 WARN ProcessingTimeExecutor: Current batch is falling behind. Th